In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pandas as pd
import numpy as np
import random

### 데이터 내려받기

In [4]:
train_df = pd.read_csv("../Data/predict_future_sales_data/sales_train.csv")
test_df = pd.read_csv("../Data/predict_future_sales_data/test.csv")
submission = pd.read_csv("../Data/predict_future_sales_data/sample_submission.csv")


In [13]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
